In Python, puoi normalizzare una serie storica di prezzi e renderla stazionaria seguendo questi passaggi:

1️⃣ Normalizzazione

La normalizzazione serve a scalare i dati tra un intervallo specifico (ad esempio 0 e 1) per rendere la serie più gestibile dai modelli di machine learning.

Ecco come puoi farlo usando MinMaxScaler di scikit-learn:

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Creiamo una serie storica di esempio
data = {'Prezzi': [100, 105, 110, 95, 115, 120]}
df = pd.DataFrame(data)

# Normalizzazione tra 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
df['Prezzi_normalizzati'] = scaler.fit_transform(df[['Prezzi']])

print(df)


   Prezzi  Prezzi_normalizzati
0     100                  0.2
1     105                  0.4
2     110                  0.6
3      95                  0.0
4     115                  0.8
5     120                  1.0


👉 Questo trasforma i dati in un intervallo tra 0 e 1, riducendo l'impatto delle variazioni di scala.

2️⃣ Stazionarietà

Una serie è stazionaria se la sua media e varianza rimangono costanti nel tempo. In finanza, le serie storiche tendono ad avere una dipendenza temporale, quindi dobbiamo eliminare tendenze o stagionalità.

✅ Un metodo comune è la differenziazione, che calcola le variazioni tra valori consecutivi:

In [2]:
df['Prezzi_differenziati'] = df['Prezzi'].diff().dropna()
print(df)

   Prezzi  Prezzi_normalizzati  Prezzi_differenziati
0     100                  0.2                   NaN
1     105                  0.4                   5.0
2     110                  0.6                   5.0
3      95                  0.0                 -15.0
4     115                  0.8                  20.0
5     120                  1.0                   5.0


👉 Questo rimuove trend e fa sì che la serie abbia una media costante nel tempo.

✅ Puoi anche verificare la stazionarietà con il test di Dickey-Fuller:

In [3]:
from statsmodels.tsa.stattools import adfuller

# Test di Dickey-Fuller
result = adfuller(df['Prezzi_differenziati'].dropna())
print(f'Statistica di test: {result[0]}')
print(f'P-value: {result[1]}')

# Se il p-value è < 0.05, la serie è stazionaria!


Statistica di test: -2.4331050121192876
P-value: 0.13260989493578812


🔥 Conclusione

Normalizzazione → MinMaxScaler() per scalare tra 0 e 1

Stazionarietà → .diff() per rimuovere trend + test ADF per conferma